In [1]:
from model import *

vocab_size = 2
context_length = 3

In [2]:
config = GPTConfig(
    block_size = context_length,
    vocab_size = vocab_size,
    n_layer = 4,
    n_head = 4,
    n_embd = 16,
    bias = False,
)
gpt = GPT(config)


number of parameters: 12656


In [3]:
# let's train our baby GPT on this sequence
seq = list(map(int, "111101111011110"))
# convert the sequence to a tensor holding all the individual examples in that sequence
X, Y = [], []
# iterate over the sequence and grab every consecutive 3 bits
# the correct label for what's next is the next bit at each position
for i in range(len(seq) - context_length):
    X.append(seq[i:i+context_length])
    Y.append(seq[i+context_length])
    print(f"example {i+1:2d}: {X[-1]} --> {Y[-1]}")
X = torch.tensor(X, dtype=torch.long)
Y = torch.tensor(Y, dtype=torch.long)
print(X.shape, Y.shape)

example  1: [1, 1, 1] --> 1
example  2: [1, 1, 1] --> 0
example  3: [1, 1, 0] --> 1
example  4: [1, 0, 1] --> 1
example  5: [0, 1, 1] --> 1
example  6: [1, 1, 1] --> 1
example  7: [1, 1, 1] --> 0
example  8: [1, 1, 0] --> 1
example  9: [1, 0, 1] --> 1
example 10: [0, 1, 1] --> 1
example 11: [1, 1, 1] --> 1
example 12: [1, 1, 1] --> 0
torch.Size([12, 3]) torch.Size([12])


In [4]:
# init a GPT and the optimizer
torch.manual_seed(1337)
optimizer = torch.optim.AdamW(gpt.parameters(), lr=1e-3, weight_decay=1e-1)

/Users/pavan/miniconda3/envs/interp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# train the GPT for some number of iterations
for i in range(500):
    logits = gpt(X)
    loss = F.cross_entropy(logits, Y)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print(i, loss.item())

0 0.466957688331604
1 0.460928350687027
2 0.46157100796699524
3 0.4533829987049103
4 0.45337602496147156
5 0.4470430314540863
6 0.4464685022830963
7 0.44136738777160645
8 0.43969476222991943
9 0.43735626339912415
10 0.43330979347229004
11 0.4323204457759857
12 0.4284249544143677
13 0.4262804090976715
14 0.42428240180015564
15 0.4209199845790863
16 0.41940978169441223
17 0.4168887436389923
18 0.41444113850593567
19 0.4129769802093506
20 0.4104703962802887
21 0.40871894359588623
22 0.40711739659309387
23 0.40494683384895325
24 0.4036227762699127
25 0.4018968641757965
26 0.40020322799682617
27 0.39899858832359314
28 0.39732322096824646
29 0.3960372507572174
30 0.3947913348674774
31 0.39333879947662354
32 0.3922746181488037
33 0.3910147249698639
34 0.3898340165615082
35 0.3888390362262726
36 0.3876636028289795
37 0.3867073059082031
38 0.3857252597808838
39 0.384717732667923
40 0.383881539106369
41 0.3829413652420044
42 0.3821183443069458
43 0.3813204765319824
44 0.3804919421672821
45 0.379

In [9]:
gpt.transformer

KeyError: 0

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Step 1: Generate random data
def generate_data(num_samples):
    data = torch.rand(num_samples, 5)
    labels = torch.sum(data, dim=1)[0]
    return data, labels

# Step 2: Define the Neural Network
class MaxOfFiveNN(nn.Module):
    def __init__(self):
        super(MaxOfFiveNN, self).__init__()
        self.fc1 = nn.Linear(5, 20)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(20, 1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Step 3: Training the network
def train_network(net, data, labels, epochs=500):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=0.01)

    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = net(data)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

        if epoch % 100 == 99:
            print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Step 4: Testing the network
def test_network(net, num_samples=10):
    test_data, test_labels = generate_data(num_samples)
    predictions = net(test_data).squeeze()
    for i in range(num_samples):
        print(f'Input: {test_data[i].numpy()}, Predicted Max: {predictions[i].item()}, Actual Max: {test_labels[i].item()}')

# Generate training data
train_data, train_labels = generate_data(10000)

# Initialize the network
net = MaxOfFiveNN()

# Train the network
train_network(net, train_data, train_labels)

# Test the network
test_network(net)


/Users/pavan/miniconda3/envs/interp/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([10000])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 100, Loss: 0.03289323300123215
Epoch 200, Loss: 0.0001919540809467435
Epoch 300, Loss: 0.00010136722266906872
Epoch 400, Loss: 6.29323476459831e-05
Epoch 500, Loss: 4.286824696464464e-05


IndexError: invalid index of a 0-dim tensor. Use `tensor.item()` in Python or `tensor.item<T>()` in C++ to convert a 0-dim tensor to a number

In [22]:
net.fc1.weight.T[0]

tensor([-0.4160, -0.2798, -0.1997,  0.4904, -0.0972,  0.2541, -0.2735, -0.4346,
         0.5342, -0.0587,  0.0415, -0.4648, -0.2574,  0.1609, -0.2753,  0.1697,
         0.2667, -0.1530,  0.1451,  0.4873], grad_fn=<SelectBackward0>)